In [1]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from get_data import get_data
import numpy as np

In [2]:
total_dt = get_data()

In [34]:
df_new = total_dt[[
     'lactation_1', 'lactation_2',  'total_production_1', 'total_production_2', 'days_milk1', 
    'days_milk2', 'mast_1',
       'mast_2',
       'inse_1', 'inse_2',
       'ab_1', 'ab_2',
    'production_ok'
]]

In [35]:
df_new.loc[df_new['total_production_1'] == 0, 'total_production_1'] = 1
df_new.loc[df_new['total_production_2'] == 0, 'total_production_2'] = 1
df_new['diff'] = round(((df_new['total_production_1'] / df_new['total_production_2'])*-100)+100,2)

/home/adrian/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/adrian/.local/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/home/adrian/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [36]:
df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
df_new.fillna(0)

/home/adrian/.local/lib/python3.8/site-packages/pandas/core/frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,lactation_1,lactation_2,total_production_1,total_production_2,days_milk1,days_milk2,mast_1,mast_2,inse_1,inse_2,ab_1,ab_2,production_ok,diff
0,1,1,7817,9123,288,319,0,0,1,3,0,0,0,14.32
1,1,1,7793,10019,290,322,0,0,1,3,0,0,0,22.22
2,1,1,10363,10293,310,392,0,0,2,2,0,0,0,-0.68
3,1,1,9367,11250,334,403,0,0,2,5,0,0,0,16.74
4,1,1,11491,12331,343,301,0,0,4,2,0,0,1,6.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378,1,1,8095,1,311,11,0,0,2,0,0,0,0,-809400.00
4379,1,1,1992,7432,58,268,0,0,1,0,0,0,0,73.20
4380,1,1,7880,1,308,9,0,0,2,0,0,0,0,-787900.00
4381,1,1,8414,2098,266,48,0,0,1,0,0,0,0,-301.05


In [7]:
df_new['diff'].value_counts()

 99.99     38
 11.53      3
-1.25       3
 6.06       3
 19.99      3
           ..
 6.54       1
 44.79      1
-4.98       1
-68.55      1
-243.12     1
Name: diff, Length: 2417, dtype: int64

In [8]:
# Define the features set.
X = df_new.copy()
X = X.drop("production_ok", axis=1)
X.head()

,lactation_1,lactation_2,total_production_1,total_production_2,days_milk1,days_milk2,mast_1,mast_2,inse_1,inse_2,ab_1,ab_2,diff
0,1,1,7817,9123,288,319,0,0,1,3,0,0,14.32
1,1,1,7793,10019,290,322,0,0,1,3,0,0,22.22
2,1,1,10363,10293,310,392,0,0,2,2,0,0,-0.68
3,1,1,9367,11250,334,403,0,0,2,5,0,0,16.74
4,1,1,11491,12331,343,301,0,0,4,2,0,0,6.81


In [9]:
# Define the target set.
y = df_new["production_ok"].ravel()
y[:5]

array([0, 0, 0, 0, 1])

In [10]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [12]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [14]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [16]:
predictions

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,

In [17]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

In [18]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,460,34
Actual 1,113,65


In [19]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [20]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,460,34
Actual 1,113,65


Accuracy Score : 0.78125
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       494
           1       0.66      0.37      0.47       178

    accuracy                           0.78       672
   macro avg       0.73      0.65      0.67       672
weighted avg       0.76      0.78      0.76       672



In [21]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.        , 0.        , 0.161273  , 0.19055421, 0.1460928 ,
       0.14844758, 0.01775964, 0.03016959, 0.0631989 , 0.079546  ,
       0.01805407, 0.01403933, 0.13086489])

In [22]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.19055420564609313, 'total_production_2'),
 (0.16127300043244464, 'total_production_1'),
 (0.14844757603185507, 'days_milk2'),
 (0.14609280204691225, 'days_milk1'),
 (0.13086488809914, 'diff'),
 (0.07954599639252466, 'inse_2'),
 (0.06319890224090713, 'inse_1'),
 (0.030169592575683843, 'mast_2'),
 (0.0180540727892874, 'ab_1'),
 (0.01775963710558539, 'mast_1'),
 (0.014039326639566654, 'ab_2'),
 (0.0, 'lactation_2'),
 (0.0, 'lactation_1')]

In [23]:
# Modelanco cambiando el valor n_estimators=128 a n_estimators =500

In [24]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [25]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [26]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [27]:
predictions

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,

In [28]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

In [29]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,457,37
Actual 1,117,61


In [30]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [31]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,457,37
Actual 1,117,61


Accuracy Score : 0.7708333333333334
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       494
           1       0.62      0.34      0.44       178

    accuracy                           0.77       672
   macro avg       0.71      0.63      0.65       672
weighted avg       0.75      0.77      0.75       672



In [32]:
importances = rf_model.feature_importances_
importances

array([0.        , 0.        , 0.15904362, 0.18885326, 0.14548969,
       0.15071062, 0.01833196, 0.0287211 , 0.06333003, 0.08057794,
       0.01805822, 0.01357503, 0.13330855])

In [33]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.18885325800502886, 'total_production_2'),
 (0.15904361532711378, 'total_production_1'),
 (0.15071062094682905, 'days_milk2'),
 (0.14548968505552246, 'days_milk1'),
 (0.13330854900769565, 'diff'),
 (0.08057793717660945, 'inse_2'),
 (0.06333003097251241, 'inse_1'),
 (0.028721096743784685, 'mast_2'),
 (0.018331956882638913, 'mast_1'),
 (0.018058215245396993, 'ab_1'),
 (0.013575034636867857, 'ab_2'),
 (0.0, 'lactation_2'),
 (0.0, 'lactation_1')]